In [19]:
import jax.numpy as jnp
from jax import vmap

from jaxgp.utils import CovMatrixDD, CovMatrixFD, CovMatrixFF
from jaxgp.kernels import RBF

In [20]:
x1 = jnp.zeros((5,1))
x2 = jnp.zeros((5,2))
x3 = jnp.zeros((5,3))
x4 = jnp.zeros((5,4))

params = jnp.array((1.0, 1.0))
kernel = RBF()

In [21]:
ff1d = CovMatrixFF(x1, x1, kernel,params)
ff2d = CovMatrixFF(x2, x2, kernel,params)
ff3d = CovMatrixFF(x3, x3, kernel,params)
ff4d = CovMatrixFF(x4, x4, kernel,params)

print(ff1d.shape, ff2d.shape, ff3d.shape, ff4d.shape)

(5, 5) (5, 5) (5, 5) (5, 5)


In [22]:
fd1d = CovMatrixFD(x1, x1, kernel,params)
fd2d = CovMatrixFD(x2, x2, kernel,params)
fd3d = CovMatrixFD(x3, x3, kernel,params)
fd4d = CovMatrixFD(x4, x4, kernel,params)

print(fd1d.shape, fd2d.shape, fd3d.shape, fd4d.shape)

fd1d = vmap(jnp.ravel, in_axes=0)(CovMatrixFD(x1, x1, kernel,params))
fd2d = vmap(jnp.ravel, in_axes=0)(CovMatrixFD(x2, x2, kernel,params))
fd3d = vmap(jnp.ravel, in_axes=0)(CovMatrixFD(x3, x3, kernel,params))
fd4d = vmap(jnp.ravel, in_axes=0)(CovMatrixFD(x4, x4, kernel,params))

print(fd1d.shape, fd2d.shape, fd3d.shape, fd4d.shape)

(5, 5, 1) (5, 5, 2) (5, 5, 3) (5, 5, 4)
(5, 5) (5, 10) (5, 15) (5, 20)


In [23]:
dd1d = CovMatrixDD(x1, x1, kernel,params)
dd2d = CovMatrixDD(x2, x2, kernel,params)
dd3d = CovMatrixDD(x3, x3, kernel,params)
dd4d = CovMatrixDD(x4, x4, kernel,params)

print(dd1d.shape, dd2d.shape, dd3d.shape, dd4d.shape)

dd1d = jnp.hstack(jnp.hstack((*CovMatrixDD(x1, x1, kernel, params),)))
dd2d = jnp.hstack(jnp.hstack((*CovMatrixDD(x2, x2, kernel, params),)))
dd3d = jnp.hstack(jnp.hstack((*CovMatrixDD(x3, x3, kernel, params),)))
dd4d = jnp.hstack(jnp.hstack((*CovMatrixDD(x4, x4, kernel, params),)))

print(dd1d.shape, dd2d.shape, dd3d.shape, dd4d.shape)

(5, 5, 1, 1) (5, 5, 2, 2) (5, 5, 3, 3) (5, 5, 4, 4)
(5, 5) (10, 10) (15, 15) (20, 20)
